#  🔹 Step 1 – Intro & Business Framing

# Part 4 – Strategic Decision System (Campaign Stop / Scale)

## 1. Business Problem

Our customers run many campaigns in parallel and spend money every day.  
Some campaigns are healthy and should be **scaled up**, while others are **wasting budget** and should be **stopped or fixed**.

As the data owner, we want to build a **simple, explainable decision system** that:

- Scores each campaign.
- Flags underperforming campaigns.
- Recommends one of a few clear actions:
  - **Scale Up**
  - **Keep Running**
  - **Optimize**
  - **Stop / Reallocate**
  - **Keep Monitoring (not enough data yet)**

## 2. Success Definition (Campaign Level)

For this exercise, a **successful campaign** is defined using:

- **Conversion Rate** at lead level  
  \[
  \text{conversion_rate} = \frac{\text{converted_leads}}{\text{total_leads}}
  \]

- **Cost per Lead (CPL)**  
  \[
  \text{CPL} = \frac{\text{total_spend}}{\text{total_leads}}
  \]

We consider a campaign “good” if:
- Its **conversion_rate** is high **relative to other campaigns**.
- Its **CPL** is low **relative to other campaigns**.
- It has **enough volume** (leads & spend) to be statistically meaningful.

We will build a **rule-based scoring system** using these signals and apply it on all campaigns in the dataset.


# 🔹 Step 2 – Load Data & Prepare Campaign-Level Metrics

In [2]:
import pandas as pd
import numpy as np

# 1. Load raw CSVs (same as previous parts)
campaign_leads = pd.read_csv("campaign_leads.csv")
campaigns = pd.read_csv("campaigns.csv")
insights = pd.read_csv("insights.csv")
lead_status_changes = pd.read_csv("lead_status_changes.csv")

# 2. Rename IDs to be explicit
campaign_leads = campaign_leads.rename(columns={"id": "lead_id"})
campaigns = campaigns.rename(columns={"id": "campaign_id"})

# 3. Parse datetime columns
campaign_leads["added_date"] = pd.to_datetime(campaign_leads["added_date"])
insights["created_at"] = pd.to_datetime(insights["created_at"])
lead_status_changes["created_at"] = pd.to_datetime(lead_status_changes["created_at"])

# 4. Derive date-only columns
campaign_leads["date"] = campaign_leads["added_date"].dt.date
insights["date"] = insights["created_at"].dt.date

# 5. Conversion definition (same as Part 2 / 3)
conversion_statuses = [
    "DONE_DEAL",
    "ALREADY_BOUGHT",
    "RESALE_REQUEST",
    "MEETING_DONE",
    "HIGH_INTEREST",
    "QUALIFIED",
]

campaign_leads["is_converted"] = campaign_leads["lead_status"].isin(conversion_statuses)

# 6. Join leads & insights with campaign meta (project_name, user_id)
leads_with_campaign = campaign_leads.merge(
    campaigns[["campaign_id", "user_id", "project_name"]],
    on="campaign_id",
    how="left"
)

insights_with_campaign = insights.merge(
    campaigns[["campaign_id", "user_id", "project_name"]],
    on="campaign_id",
    how="left"
)

leads_with_campaign.head()


,lead_id,campaign_id,name,email,phone,lead_status,added_date,date,is_converted,user_id,project_name
0,77930,6496,Vicky Mohr Sr.,smith.oren@example.org,+2015628437879,UNKNOWN,2024-06-01 17:00:40,2024-06-01,False,318,Il Cazar Safia north coast
1,77942,6496,Clovis Mueller,toy.korey@example.net,+1-615-909-5723,UNKNOWN,2024-06-01 18:22:00,2024-06-01,False,318,Il Cazar Safia north coast
2,77964,6493,Austin Ondricka II,hveum@example.org,+2014034035897,NOT_QUALIFIED,2024-06-01 21:56:33,2024-06-01,False,466,Azha North Coast Ras El Hekma
3,77983,6500,Jaquan Kuhn,greenholt.elsa@example.com,310-242-6257,UNKNOWN,2024-06-02 01:13:19,2024-06-02,False,217,sky ad - new cairo launch
4,77988,6496,Prof. Grayson Collier,obie83@example.com,+2016402961934,UNKNOWN,2024-06-02 01:41:56,2024-06-02,False,318,Il Cazar Safia north coast


In [12]:
leads_with_campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56965 entries, 0 to 56964
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   lead_id       56965 non-null  int64         
 1   campaign_id   56965 non-null  int64         
 2   name          56965 non-null  object        
 3   email         56965 non-null  object        
 4   phone         56965 non-null  object        
 5   lead_status   56965 non-null  object        
 6   added_date    56965 non-null  datetime64[ns]
 7   date          56965 non-null  object        
 8   is_converted  56965 non-null  bool          
 9   user_id       56965 non-null  int64         
 10  project_name  56965 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(3), object(6)
memory usage: 4.4+ MB


# 🔹 Step 3 – Build Campaign-Level Performance Table

## 3. Build Campaign-Level Performance Table

We now aggregate all data at the **campaign** level.

For each campaign we will compute:

- `total_spend`
- `total_leads`
- `converted_leads`
- `conversion_rate`
- `CPL` (cost per lead)
- `days_active` (number of days with spend)
- `leads_per_day`
- `spend_per_day`

This will be the input to our decision system.


In [3]:
# 3.1 Spend per campaign
spend_per_campaign = (
    insights_with_campaign
    .groupby("campaign_id", as_index=False)
    .agg(
        total_spend=("spend", "sum"),
        days_active=("date", "nunique")
    )
)

# 3.2 Leads per campaign
leads_per_campaign = (
    leads_with_campaign
    .groupby("campaign_id", as_index=False)
    .agg(
        total_leads=("lead_id", "count"),
        converted_leads=("is_converted", "sum")
    )
)

# 3.3 Merge spend & leads, then add campaign meta
campaign_perf_4 = spend_per_campaign.merge(
    leads_per_campaign,
    on="campaign_id",
    how="outer"
)

campaign_perf_4 = campaign_perf_4.merge(
    campaigns[["campaign_id", "project_name", "user_id", "daily_budget"]],
    on="campaign_id",
    how="left"
)

# Fill NaNs
campaign_perf_4["total_spend"] = campaign_perf_4["total_spend"].fillna(0)
campaign_perf_4["days_active"] = campaign_perf_4["days_active"].fillna(0).astype(int)
campaign_perf_4["total_leads"] = campaign_perf_4["total_leads"].fillna(0).astype(int)
campaign_perf_4["converted_leads"] = campaign_perf_4["converted_leads"].fillna(0).astype(int)

# 3.4 Derived KPIs
campaign_perf_4["conversion_rate"] = np.where(
    campaign_perf_4["total_leads"] > 0,
    campaign_perf_4["converted_leads"] / campaign_perf_4["total_leads"],
    np.nan
)

campaign_perf_4["cpl"] = np.where(
    campaign_perf_4["total_leads"] > 0,
    campaign_perf_4["total_spend"] / campaign_perf_4["total_leads"],
    np.nan
)

campaign_perf_4["leads_per_day"] = np.where(
    campaign_perf_4["days_active"] > 0,
    campaign_perf_4["total_leads"] / campaign_perf_4["days_active"],
    np.nan
)

campaign_perf_4["spend_per_day"] = np.where(
    campaign_perf_4["days_active"] > 0,
    campaign_perf_4["total_spend"] / campaign_perf_4["days_active"],
    np.nan
)

campaign_perf_4.head()


,campaign_id,total_spend,days_active,total_leads,converted_leads,project_name,user_id,daily_budget,conversion_rate,cpl,leads_per_day,spend_per_day
0,6493,689.19,2,1,0,Azha North Coast Ras El Hekma,466,1200,0.0,689.190,0.500000,344.595
1,6496,2137.86,3,4,0,Il Cazar Safia north coast,318,2000,0.0,534.465,1.333333,712.620
2,6497,888.43,2,1,0,Plage,514,1200,0.0,888.430,0.500000,444.215
3,6499,354.40,2,0,0,Plage,514,1200,NaN,NaN,0.000000,177.200
4,6500,842.40,2,1,0,sky ad - new cairo launch,217,1200,0.0,842.400,0.500000,421.200


In [13]:
campaign_perf_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823 entries, 0 to 6822
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_id      6823 non-null   int64  
 1   total_spend      6823 non-null   float64
 2   days_active      6823 non-null   int64  
 3   total_leads      6823 non-null   int64  
 4   converted_leads  6823 non-null   int64  
 5   project_name     6823 non-null   object 
 6   user_id          6823 non-null   int64  
 7   daily_budget     6823 non-null   int64  
 8   conversion_rate  4022 non-null   float64
 9   cpl              4022 non-null   float64
 10  leads_per_day    6821 non-null   float64
 11  spend_per_day    6821 non-null   float64
dtypes: float64(5), int64(6), object(1)
memory usage: 639.8+ KB


# 🔹 Step 4– Explore Distributions & Benchmarks

## 4. Explore Distributions & Global Benchmarks

Before defining rules, we inspect how campaigns behave:

- How many campaigns have **enough data** (leads & spend)?
- What are typical values of:
  - conversion_rate
  - CPL

We compute percentiles to define **high/low** thresholds.


In [4]:
# Filter campaigns that have at least some spend and leads
mask_has_data = (campaign_perf_4["total_spend"] > 0) & (campaign_perf_4["total_leads"] >= 10)
perf_non_empty = campaign_perf_4[mask_has_data].copy()

print("Number of campaigns with >=10 leads and >0 spend:", len(perf_non_empty))

# Basic statistics
perf_non_empty[["conversion_rate", "cpl", "leads_per_day", "spend_per_day"]].describe()


Number of campaigns with >=10 leads and >0 spend: 1031


,conversion_rate,cpl,leads_per_day,spend_per_day
count,1031.000000,1031.000000,1031.000000,1031.000000
mean,0.063029,373.463914,4.503198,1377.187359
std,0.121731,498.835189,6.705336,2078.256982
min,0.000000,3.710809,0.916667,205.895000
25%,0.000000,215.929276,2.000000,619.766000
50%,0.000000,336.720714,3.000000,831.199091
75%,0.076923,459.091556,4.818182,1335.667556
max,1.000000,12595.661512,136.000000,27798.308333


In [14]:
perf_non_empty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031 entries, 20 to 6821
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_id      1031 non-null   int64  
 1   total_spend      1031 non-null   float64
 2   days_active      1031 non-null   int64  
 3   total_leads      1031 non-null   int64  
 4   converted_leads  1031 non-null   int64  
 5   project_name     1031 non-null   object 
 6   user_id          1031 non-null   int64  
 7   daily_budget     1031 non-null   int64  
 8   conversion_rate  1031 non-null   float64
 9   cpl              1031 non-null   float64
 10  leads_per_day    1031 non-null   float64
 11  spend_per_day    1031 non-null   float64
dtypes: float64(5), int64(6), object(1)
memory usage: 104.7+ KB


In [5]:
conv_p25, conv_p50, conv_p75 = np.nanpercentile(
    perf_non_empty["conversion_rate"].dropna(),
    [25, 50, 75]
)

cpl_p25, cpl_p50, cpl_p75 = np.nanpercentile(
    perf_non_empty["cpl"].dropna(),
    [25, 50, 75]
)

print("Conversion Rate percentiles:")
print("P25:", conv_p25, " | P50:", conv_p50, " | P75:", conv_p75)

print("\nCPL percentiles:")
print("P25:", cpl_p25, " | P50:", cpl_p50, " | P75:", cpl_p75)


Conversion Rate percentiles:
P25: 0.0  | P50: 0.0  | P75: 0.07692307692307693

CPL percentiles:
P25: 215.92927631578948  | P50: 336.7207142857143  | P75: 459.09155555555554


# 🔹 Step 5 – Define Rule-Based Decision System

## 5. Rule-Based Campaign Decision System

We define a simple, explainable rule-based system.

### Signals used:

- `total_leads` – volume of leads
- `total_spend` – money invested
- `conversion_rate` – effectiveness
- `cpl` – cost efficiency

### Rules (high-level):

1. **Not enough data**  
   - Few leads or very low spend → we cannot judge yet  
   → Action: `Keep Monitoring`

2. **Hero campaigns (High Conv – Low CPL)**  
   - conversion_rate ≥ P75  
   - cpl ≤ P50  
   → Action: `Scale Up`

3. **Good campaigns (OK Conv – Acceptable CPL)**  
   - conversion_rate ≥ P50  
   - cpl ≤ P75  
   → Action: `Keep Running`

4. **Poor campaigns (Low Conv – High CPL)**  
   - conversion_rate < P25  
   - cpl > P75  
   → Action: `Stop / Reallocate`

5. **In-between**  
   - Not clearly good or bad  
   → Action: `Optimize` (targets, creatives, messaging)


In [7]:
# Hyperparameters for "enough data"
MIN_LEADS = 30
MIN_SPEND = 200.0  # adjust later with business

def score_single_campaign(row):
    """
    Takes a row from campaign_perf_4 and returns:
    - health_bucket
    - recommended_action
    """
    tl = row["total_leads"]
    ts = row["total_spend"]
    cr = row["conversion_rate"]
    cpl = row["cpl"]
    
    # 1) Not enough data
    if (tl < MIN_LEADS) or (ts < MIN_SPEND) or pd.isna(cr) or pd.isna(cpl):
        return pd.Series({
            "health_bucket": "NEEDS_MORE_DATA",
            "recommended_action": "Keep Monitoring"
        })
    
    # 2) Hero campaigns (High Conv – Low CPL)
    if (cr >= conv_p75) and (cpl <= cpl_p50):
        return pd.Series({
            "health_bucket": "HERO",
            "recommended_action": "Scale Up"
        })
    
    # 3) Good campaigns (OK Conv – Acceptable CPL)
    if (cr >= conv_p50) and (cpl <= cpl_p75):
        return pd.Series({
            "health_bucket": "GOOD",
            "recommended_action": "Keep Running"
        })
    
    # 4) Poor campaigns (Low Conv – High CPL)
    if (cr < conv_p25) and (cpl > cpl_p75):
        return pd.Series({
            "health_bucket": "POOR",
            "recommended_action": "Stop / Reallocate"
        })
    
    # 5) Mixed cases → need optimization
    return pd.Series({
        "health_bucket": "MIXED",
        "recommended_action": "Optimize (Targeting / Creative / Audience)"
    })


# Apply on all campaigns
scores = campaign_perf_4.apply(score_single_campaign, axis=1)
campaign_scored = pd.concat([campaign_perf_4, scores], axis=1)

campaign_scored.head()


,campaign_id,total_spend,days_active,total_leads,converted_leads,project_name,user_id,daily_budget,conversion_rate,cpl,leads_per_day,spend_per_day,health_bucket,recommended_action
0,6493,689.19,2,1,0,Azha North Coast Ras El Hekma,466,1200,0.0,689.190,0.500000,344.595,NEEDS_MORE_DATA,Keep Monitoring
1,6496,2137.86,3,4,0,Il Cazar Safia north coast,318,2000,0.0,534.465,1.333333,712.620,NEEDS_MORE_DATA,Keep Monitoring
2,6497,888.43,2,1,0,Plage,514,1200,0.0,888.430,0.500000,444.215,NEEDS_MORE_DATA,Keep Monitoring
3,6499,354.40,2,0,0,Plage,514,1200,NaN,NaN,0.000000,177.200,NEEDS_MORE_DATA,Keep Monitoring
4,6500,842.40,2,1,0,sky ad - new cairo launch,217,1200,0.0,842.400,0.500000,421.200,NEEDS_MORE_DATA,Keep Monitoring


In [15]:
campaign_scored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6823 entries, 0 to 6822
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   campaign_id         6823 non-null   int64  
 1   total_spend         6823 non-null   float64
 2   days_active         6823 non-null   int64  
 3   total_leads         6823 non-null   int64  
 4   converted_leads     6823 non-null   int64  
 5   project_name        6823 non-null   object 
 6   user_id             6823 non-null   int64  
 7   daily_budget        6823 non-null   int64  
 8   conversion_rate     4022 non-null   float64
 9   cpl                 4022 non-null   float64
 10  leads_per_day       6821 non-null   float64
 11  spend_per_day       6821 non-null   float64
 12  health_bucket       6823 non-null   object 
 13  recommended_action  6823 non-null   object 
dtypes: float64(5), int64(6), object(3)
memory usage: 746.4+ KB


# 🔹 Step 6 – Summary: How Many Campaigns Fall into Each Action?

## 6. Portfolio View – How Many Campaigns in Each Bucket?

We now look at the distribution of campaigns across:

- `NEEDS_MORE_DATA`
- `HERO`
- `GOOD`
- `POOR`
- `MIXED`

This tells us how many campaigns are candidates to **Scale**, **Stop**, or **Optimize**.


In [8]:
bucket_counts = campaign_scored["health_bucket"].value_counts(dropna=False)
action_counts = campaign_scored["recommended_action"].value_counts(dropna=False)

print("Health buckets:")
print(bucket_counts)

print("\nRecommended actions:")
print(action_counts)


Health buckets:
health_bucket
NEEDS_MORE_DATA    6341
GOOD                336
MIXED                88
HERO                 58
Name: count, dtype: int64

Recommended actions:
recommended_action
Keep Monitoring                               6341
Keep Running                                   336
Optimize (Targeting / Creative / Audience)      88
Scale Up                                        58
Name: count, dtype: int64


In [9]:
action_summary = (
    campaign_scored
    .groupby(["health_bucket", "recommended_action"], as_index=False)
    .agg(
        n_campaigns=("campaign_id", "nunique"),
        total_spend=("total_spend", "sum")
    )
    .sort_values("n_campaigns", ascending=False)
)

action_summary


,health_bucket,recommended_action,n_campaigns,total_spend
3,NEEDS_MORE_DATA,Keep Monitoring,6341,9945778.10
0,GOOD,Keep Running,336,6964058.75
2,MIXED,Optimize (Targeting / Creative / Audience),88,6655157.41
1,HERO,Scale Up,58,1287297.33


# 🔹 Step 6 – Apply System to Real Campaigns (Examples)

In [10]:
scale_up = (
    campaign_scored
    [campaign_scored["recommended_action"] == "Scale Up"]
    .sort_values("conversion_rate", ascending=False)
    .head(10)
    .copy()
)

scale_up[[
    "campaign_id", "project_name", "user_id",
    "total_spend", "total_leads", "converted_leads",
    "conversion_rate", "cpl", "health_bucket", "recommended_action"
]]


,campaign_id,project_name,user_id,total_spend,total_leads,converted_leads,conversion_rate,cpl,health_bucket,recommended_action
1571,8521,Veranda Sahl Hasheesh,993,6504.34,99,99,1.000000,65.700404,HERO,Scale Up
1755,8770,Veranda Sahl Hasheesh,993,4850.94,48,48,1.000000,101.061250,HERO,Scale Up
1570,8520,Veranda Sahl Hasheesh,993,1705.48,38,37,0.973684,44.881053,HERO,Scale Up
698,7382,Silversands - Ora Developments,349,6368.00,61,30,0.491803,104.393443,HERO,Scale Up
76,6605,Silversands - Ora Developments,349,7232.67,47,23,0.489362,153.886596,HERO,Scale Up
1053,7860,Trio Gardens,132,10630.47,76,36,0.473684,139.874605,HERO,Scale Up
1238,8103,Plage,976,3164.73,35,16,0.457143,90.420857,HERO,Scale Up
906,7661,Belle Vie,49,6971.52,180,82,0.455556,38.730667,HERO,Scale Up
823,7543,Mar Bay - Al Marasem Ras El Hekma,841,11771.44,57,23,0.403509,206.516491,HERO,Scale Up
567,7207,Mountain View iCity October,199,17389.16,85,34,0.400000,204.578353,HERO,Scale Up


In [11]:
stop_reallocate = (
    campaign_scored
    [campaign_scored["recommended_action"] == "Stop / Reallocate"]
    .sort_values("cpl", ascending=False)
    .head(10)
    .copy()
)

stop_reallocate[[
    "campaign_id", "project_name", "user_id",
    "total_spend", "total_leads", "converted_leads",
    "conversion_rate", "cpl", "health_bucket", "recommended_action"
]]


,campaign_id,project_name,user_id,total_spend,total_leads,converted_leads,conversion_rate,cpl,health_bucket,recommended_action


# 🔹 Step 7 – Assumptions & Questions for Stakeholders 

## 8. Assumptions & Questions for Stakeholders

### Assumptions

1. **Conversion Definition**
   - A lead is considered "converted" if its final status is in:  
     `["DONE_DEAL", "ALREADY_BOUGHT", "RESALE_REQUEST", "MEETING_DONE", "HIGH_INTEREST", "QUALIFIED"]`.
   - We assume these statuses are stable and correctly maintained by the sales team.

2. **Data Quality**
   - We assume that spend and leads are correctly attributed to campaigns.
   - We ignore campaigns with very low spend or very few leads when judging performance.

3. **Thresholds**
   - We use **percentiles (P25, P50, P75)** of conversion_rate and CPL to define "high" and "low".
   - We assume `MIN_LEADS = 30` and `MIN_SPEND = 200` are reasonable thresholds for "enough data".

4. **Business Objective**
   - The main objective is **cost-efficient conversions**, not just volume of leads.

### Questions to Clarify with Business

1. **Conversion Definition**
   - Do you agree with our list of statuses for "converted"?  
     Should we include/exclude any statuses?

2. **Minimum Volume**
   - What is the **minimum number of leads** and **minimum spend** you trust to judge a campaign?

3. **Risk Appetite**
   - Are you comfortable stopping campaigns based purely on CPL and conversion_rate?
   - Do you prefer a **"soft" stop** (optimize first) before hard stopping?

4. **Action Labels**
   - Are the actions `Scale Up / Keep Running / Optimize / Stop / Keep Monitoring` intuitive for your sales & marketing teams?

5. **Delivery Mechanism**
   - Would you prefer this system:
     - As part of the main dashboard?
     - As a weekly PDF/email report?
     - As alerts (e.g. “these 5 campaigns need action this week”)?


In [16]:
def classify_campaign(row, conv_p75, cpl_p25, cpl_p50, cpl_p75):
    conv = row["conversion_rate"]
    cpl  = row["cpl"]
    leads = row["total_leads"]
    spend = row["total_spend"]
    days  = row["days_active"]

    # 1) Not enough data
    if (leads < 10) or (spend <= 0) or (days < 3):
        return "NEEDS_MORE_DATA", "Keep Monitoring"

    # 2) BAD – pure waste: high CPL, zero conv, enough data
    if (leads >= 20) and (conv == 0) and (cpl >= cpl_p75):
        return "BAD", "Stop / Reallocate"

    # 3) HERO – high conv, low CPL
    if (conv >= conv_p75) and (cpl <= cpl_p25):
        return "HERO", "Scale Up"

    # 4) GOOD – decent conv, acceptable CPL
    if (conv >= conv_p75) and (cpl <= cpl_p50):
        return "GOOD", "Keep Running"

    # 5) MIXED – either conv ok بس غالي، أو conv ضعيف بس رخيص
    if (conv > 0) and (cpl > cpl_p50):
        return "MIXED", "Optimize (Targeting / Creative / Audience)"

    # 6) Default → نسيبها مونيتور
    return "NEEDS_MORE_DATA", "Keep Monitoring"


In [17]:
campaign_scored[["health_bucket", "recommended_action"]] = campaign_scored.apply(
    lambda row: pd.Series(
        classify_campaign(row, conv_p75, cpl_p25, cpl_p50, cpl_p75)
    ),
    axis=1
)


In [18]:
stop_reallocate = (
    campaign_scored
    [campaign_scored["recommended_action"] == "Stop / Reallocate"]
    .sort_values("cpl", ascending=False)
    .head(10)
    .copy()
)

stop_reallocate[
    [
        "campaign_id", "project_name", "user_id",
        "total_spend", "total_leads", "converted_leads",
        "conversion_rate", "cpl", "health_bucket", "recommended_action"
    ]
]


,campaign_id,project_name,user_id,total_spend,total_leads,converted_leads,conversion_rate,cpl,health_bucket,recommended_action
6805,15208,HPD-Voke Mall,1635,1083226.89,86,0,0.0,12595.661512,BAD,Stop / Reallocate
6815,15228,New Cairo Villas General,1635,1246692.60,144,0,0.0,8657.587500,BAD,Stop / Reallocate
4748,12685,Mountain View iCity October,411,41571.20,29,0,0.0,1433.489655,BAD,Stop / Reallocate
1715,8717,Dubai General,1020,39927.00,28,0,0.0,1425.964286,BAD,Stop / Reallocate
4743,12680,Mountain View iCity October,411,42093.92,34,0,0.0,1238.056471,BAD,Stop / Reallocate
5948,14159,Ivoire West - PRE,1535,68870.28,67,0,0.0,1027.914627,BAD,Stop / Reallocate
6415,14707,PX by Palm Hills,1701,23913.11,24,0,0.0,996.379583,BAD,Stop / Reallocate
1842,8882,Makadi Heights,1030,22053.11,25,0,0.0,882.124400,BAD,Stop / Reallocate
2280,9450,Mountain View Hyde Park,551,20082.97,23,0,0.0,873.172609,BAD,Stop / Reallocate
4413,12271,Kings Way,1322,16444.99,20,0,0.0,822.249500,BAD,Stop / Reallocate


# Documents

# Part 4 – Strategic Decision System  
**“Should this campaign continue or stop?”**

In this part, I build a **rule-based decision system** on top of the Leadsmart data to help the business decide:

- Which campaigns should **keep running**?
- Which campaigns should be **scaled up**?
- Which campaigns should be **optimized**?
- Which campaigns should be **stopped or reallocated**?
- And when is there **not enough data** to make a decision?

The goal is to:

- Define a **data-driven notion of success** per campaign.
- Use **platform-level benchmarks** instead of arbitrary thresholds.
- Produce a clear **`recommended_action`** for every campaign:
  - `Scale Up`
  - `Keep Running`
  - `Optimize (Targeting / Creative / Audience)`
  - `Stop / Reallocate`
  - `Keep Monitoring`


# 1. campaign_leads.csv

Each row represents one lead generated by a campaign.
We used the following key fields:

Column	Meaning
lead_id	Unique identifier for the lead
campaign_id	The campaign from which the lead originated
lead_status	Status of the lead (NEW_LEAD, QUALIFIED, DONE_DEAL, etc.)
added_date	Timestamp when the lead was captured
user_id	The client / partner associated with the campaign
project_name	The real-estate project the campaign belongs to

## 1. Data Used in the Decision System

The system is built on top of the **same Leadsmart datasets** used in previous parts:

### 1.1 `campaign_leads.csv`

Each row = **one lead** generated by a campaign.

Key columns:

- `lead_id` – unique identifier for the lead  
- `campaign_id` – which campaign generated this lead  
- `lead_status` – status of the lead  
  - e.g. `NEW_LEAD`, `QUALIFIED`, `DONE_DEAL`, `MEETING_DONE`, `NO_ANSWER`, ...  
- `added_date` – when the lead was captured  
- `user_id` – the client / partner  
- `project_name` – the real-estate project name

---

### 1.2 `campaigns.csv`

Metadata about each campaign.

Columns used:

- `campaign_id`  
- `user_id`  
- `project_name`  
- `daily_budget` – expected daily budget for the campaign

---

### 1.3 `insights.csv`

Daily performance data from the ad platforms.

Columns used:

- `campaign_id`  
- `spend` – daily spend  
- `created_at` – timestamp of the spend event

From this table we derive:

- `total_spend` per campaign  
- `days_active` per campaign (number of days with activity / spend)


## 2. Conversion Definition

To decide whether a campaign is successful, we need a **clear definition of conversion** at the lead level.

I defined a lead as **converted** if its `lead_status` is in the following set:

```python
conversion_statuses = [
    "DONE_DEAL",
    "ALREADY_BOUGHT",
    "RESALE_REQUEST",
    "MEETING_DONE",
    "HIGH_INTEREST",
    "QUALIFIED",
]


---

## 🧱 Cell 4 – Campaign-Level Table

### 3. Campaign-Level Performance Table

In this step, we aggregate the three datasets into a **campaign-level table**  
(one row = one campaign). We refer to this table as **`campaign_perf_4`**.

---

### 3.1 Metrics from `insights.csv`

For each `campaign_id`:

- **total_spend**: sum of all `spend` values across days  
- **days_active**: number of distinct days with activity/spend  

---

### 3.2 Metrics from `campaign_leads.csv`

For each `campaign_id`:

- **total_leads**: count of all leads  
- **converted_leads**: count of leads where `is_converted = True`  

---

### 3.3 Metadata from `campaigns.csv`

We attach the following metadata:

- `project_name`  
- `user_id`  
- `daily_budget`

---

### 3.4 Engineered KPIs (per campaign)

#### **Conversion Rate**
\[
\text{conversion_rate} =
\begin{cases}
\dfrac{\text{converted\_leads}}{\text{total\_leads}}, & \text{if total\_leads > 0} \\[4pt]
0, & \text{otherwise}
\end{cases}
\]

---

#### **CPL (Cost per Lead)**
\[
\text{cpl} =
\begin{cases}
\dfrac{\text{total\_spend}}{\text{total\_leads}}, & \text{if total\_leads > 0} \\[4pt]
\text{NaN}, & \text{otherwise}
\end{cases}
\]

> Note: CPL remains **NaN** when there are no leads to avoid treating it as “cheap”.

---

#### **Leads per Day**
\[
\text{leads\_per\_day} = \dfrac{\text{total\_leads}}{\text{days\_active}}
\]

---

#### **Spend per Day**
\[
\text{spend\_per\_day} = \dfrac{\text{total\_spend}}{\text{days\_active}}
\]

---

### Final Table Structure

Each campaign row includes:

`campaign_id`, `project_name`, `user_id`, `daily_budget`,  
`total_spend`, `days_active`, `total_leads`, `converted_leads`,  
`conversion_rate`, `cpl`, `leads_per_day`, `spend_per_day`.

---

This table is the **core input** for the decision system.

---


## 4. Benchmarking: Where Does a Campaign Stand vs the Platform?

Instead of hard-coding thresholds (e.g. “5% conversion is good”),  
I use the **distribution of all campaigns** on the platform to define what “good” and “bad” mean.

### 4.1 Selecting Reliable Benchmark Campaigns

To avoid noisy campaigns with very few leads, I filter to:

- `total_leads >= 10`
- `total_spend > 0`

The resulting subset (named `perf_non_empty`) contains **≈ 1,031 campaigns**  
and is used as the benchmarking universe.

---

### 4.2 Percentiles for Conversion Rate

Based on `perf_non_empty["conversion_rate"]`:

- **P25** = 0.0  
- **P50** = 0.0  
- **P75** ≈ 0.0769  → ~7.7%

**Insight:**  
Most campaigns do **not** generate any conversions.  
Only the top ~25% manage to reach a conversion rate above ~7–8%.

---

### 4.3 Percentiles for CPL (Cost per Lead)

Based on `perf_non_empty["cpl"]`:

- **P25** ≈ 216  
- **P50** ≈ 337  
- **P75** ≈ 459  

Interpretation:

- `cpl < 216`   → among the **cheapest 25%** of campaigns  
- `cpl > 459`   → among the **most expensive 25%**  

These percentiles become the **thresholds** used to classify campaigns into health buckets.


## 5. Health Buckets & Decision Rules

I define a function `classify_campaign(row)` that assigns two fields:

- `health_bucket`        → overall health label  
- `recommended_action`   → what the business should do next  

The logic is **fully explainable** and derived from the percentiles above.

---

### 5.1 Bucket 1 – `NEEDS_MORE_DATA`

First question: *“Do we have enough data to judge this campaign?”*  

A campaign is marked as **NEEDS_MORE_DATA** if **any** of the following hold:

- `total_leads < 10`, OR  
- `total_spend <= 0`, OR  
- `days_active < 3`

Decision:

- `health_bucket = "NEEDS_MORE_DATA"`  
- `recommended_action = "Keep Monitoring"`

**Business meaning:**  
The campaign is still very young or low-volume. We shouldn’t stop or scale it yet.  
We simply keep monitoring until it gathers more evidence.
---

### 5.2 Bucket 2 – `BAD` (Stop / Reallocate)

A **clearly bad** campaign must satisfy all of:

- `total_leads >= 20`              → enough volume  
- `conversion_rate == 0`           → zero conversions  
- `cpl >= cpl_p75`                 → in the worst 25% of CPL

Decision:

- `health_bucket = "BAD"`  
- `recommended_action = "Stop / Reallocate"`

**Business meaning:**  
This campaign is:

- Spending real money  
- Generating leads  
- Converting **none** of them  
- And doing so at a **high cost**

These are the **first campaigns to switch off or reallocate budget from.**
---

### 5.3 Bucket 3 – `HERO` (Scale Up)

A **hero** campaign is both highly efficient and highly effective:

- `conversion_rate >= conv_p75`    → among the best 25% in conversion rate  
- `cpl <= cpl_p25`                 → among the cheapest 25% in CPL

Decision:

- `health_bucket = "HERO"`  
- `recommended_action = "Scale Up"`

**Business meaning:**  
This is top-tier performance: strong conversion at low cost.  
It deserves **more budget** and can be used as a **best-practice example**.

---

### 5.4 Bucket 4 – `GOOD` (Keep Running)

Some campaigns are solid but not at the HERO level:

- `conversion_rate >= conv_p75`    → high conversion  
- `cpl <= cpl_p50`                 → CPL not in the worst half of the market  

Decision:

- `health_bucket = "GOOD"`  
- `recommended_action = "Keep Running"`

**Business meaning:**  
These campaigns are performing well and should keep running.  
We may test creative / targeting tweaks, but there is no urgent need to change.

---

### 5.5 Bucket 5 – `MIXED` (Optimize)

Here we focus on campaigns that:

- **do** generate conversions (`conversion_rate > 0`),  
- but do so at **above-average cost** (`cpl > cpl_p50`).

Decision:

- `health_bucket = "MIXED"`  
- `recommended_action = "Optimize (Targeting / Creative / Audience)"`

**Business meaning:**  
The campaign works, but it is **too expensive**. Before scaling, we should:

- Refine targeting  
- Refresh or test creatives  
- Revisit audience and placement strategy  

---

### 5.6 Default Case

If a campaign does not clearly fall into `BAD`, `HERO`, `GOOD`, or `MIXED`,  
and the data is still ambiguous, it is classified back as:

- `health_bucket = "NEEDS_MORE_DATA"`  
- `recommended_action = "Keep Monitoring"`

This avoids over-reacting to noisy or borderline cases.


## 6. What the System Finds on Real Data

When applying these rules on the real campaigns, we obtain:

- A large group of campaigns in `NEEDS_MORE_DATA` (young / low volume)
- A subset of `GOOD` and `HERO` campaigns that are genuinely efficient
- A smaller but very important group of `BAD` campaigns:
  - high spend
  - many leads
  - **zero conversions**
  - very high CPL  
- A number of `MIXED` campaigns that convert but at high cost

At the portfolio level, we can summarise:

- How much budget is currently flowing into **HERO / GOOD** vs **BAD / MIXED / NEEDS_MORE_DATA**
- Which clients or projects have the largest concentration of **HERO** campaigns
- Which campaigns are the **top candidates to pause / reallocate**.


## 7. From Rules to a Recommender System

The rule engine outputs, for every campaign:

- `health_bucket`
- `recommended_action`

In the **Streamlit dashboard**, this is surfaced through three main views:

---

### 7.1 Overview tab

- Shows:
  - data window (`min_date` → `max_date`)
  - definitions of conversion and KPIs
- KPI cards:
  - total number of campaigns
  - number of `HERO`, `GOOD`, `MIXED`, `BAD`, `NEEDS_MORE_DATA`
- Summary table:
  - for each `recommended_action`:
    - number of campaigns
    - total spend

This answers:

> “How is my budget distributed across good vs bad campaigns?”

---

### 7.2 Campaign Explorer tab

- Sidebar filters for:
  - `user_id`
  - `project_name`
  - `recommended_action`
- Visuals:
  - Bar chart of campaign counts by `health_bucket`
  - Scatter plot of `cpl` vs `conversion_rate` coloured by `health_bucket`
- Detailed table:
  - For each filtered campaign: full metrics + `health_bucket` + `recommended_action`

This answers:

> “For this client or project, which campaigns are HERO / GOOD / BAD?”

---

### 7.3 Single Campaign Recommendation tab

- User selects a `campaign_id` from a dropdown
- The app displays:
  - `project_name`, `user_id`
  - `total_spend`, `days_active`
  - `total_leads`, `converted_leads`
  - `conversion_rate`, `cpl`
  - `health_bucket`
  - `recommended_action`
  - **Plain-English explanation** of *why* this decision was made

Examples:

- **BAD**  
  > “High spend, enough leads, zero conversions, and CPL in the worst 25% →  
  > *Recommendation: Stop / Reallocate.*”

- **HERO**  
  > “Conversion rate in the top 25% and CPL in the cheapest 25% →  
  > *Recommendation: Scale Up.*”

This turns the rule engine into a **practical decision support tool** that a non-technical business user can rely on every day.


## 8. Assumptions & Questions for Stakeholders

### 8.1 Key Assumptions

- The list of `conversion_statuses` correctly represents a **successful lead**.
- All campaigns in the benchmark window share a **similar business context**
  (same market, similar product type).
- The chosen thresholds:
  - `total_leads >= 10`
  - `days_active >= 3`
  - `total_spend > 0`
  are acceptable as a definition of “enough data”.
- It is acceptable to use **platform-wide percentiles** (P25 / P50 / P75) as a proxy
  for “good” vs “bad” performance.

### 8.2 Questions to Clarify in a Real Engagement

- Should different **verticals / project types** have different thresholds?
- Are there additional **business events** (e.g. offline sales) that should be
  included in the conversion definition?
- Is the business more sensitive to:
  - *wasting money* (high CPL, zero conversions), or
  - *missed opportunity* (good conversion but low budget)?
- What is the **minimum time window** for making a stop / continue decision?
  (e.g. 7 days, 14 days, 30 days)

Answering these questions would allow us to:

- Tune the thresholds per client segment
- Integrate more signals (e.g. revenue, profit, LTV)
- Evolve this rule-based system into a **full probabilistic or ML-based policy**.
